##  - Predicting stock price is tricky. 
##  - THis is like a Time series problem.
##  - Based on previous stock prices predicting the future stock price is not possible. 
##  - But usigt the previous stock price we can predict the trend
##  - i.e We can predict wheather the trend is Upward or downward

# Part 1 - Data Preprocessing

In [5]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [6]:
# Importing the training set
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
# we will load only training data set using this set we will train the model and use test data for predicting
# Also the trained RNN Model will have no idea about test set

In [7]:
training_set = dataset_train.iloc[:, 1:2].values
# Once loading training data we have to take the right column to bild model
# Here we need only "Open" column from dataset. "open" measn stock price opened at that particular price for that day
# [1:2] because we want to create an array of "Open" column in python upperbound(2nd column) is ignored
training_set

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

# We need to create NUMPY array of the column beacuse only Numpy array can be the input of Neural Network in Keras

#  - Feature Scaling 
### - THe two best ways of applying feature scaling is:-
## 1 : Standerdization :  X(stand) = (X - Mean(X) / (std.dev(X))
## 2 : Noramalization  : (X - Min(X) / (Max(X) - Min(X)) 

## Whenever we build RNN and espically we have sigmoid function in output layer of RNN, its better to use Normalization In RNN

In [4]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

In [5]:
sc = MinMaxScaler(feature_range = (0, 1)) 
# All scaled values will be between 0 and 1

In [6]:
# Applying SC object created on our dataset
training_set_scaled = sc.fit_transform(training_set)
# fit.transform applies Normalization formula to each observation
#scaled data 
training_set_scaled

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

# -- Most imp step of data preprocesssing in RNN 
# 1. Create a timestep
### - It is imp to have right number of time steps otherwise there will be overfitting

## -  We will create a timestep of 60 andd 1 output
### - At each time T, RNN will look at 60 stock prizes before time T(60)
### - And based on trends, it has captured during these 60 stock prizes(observations) it will try to predict the next output

### - Here 60 time step - 0th - 59th observation and 1 output - 60th observation

In [7]:
### First we will reate two empty entities
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []

In [8]:
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
# As the time step suitable here is 60, In order to get 60 previous stock prize we need to start with 60th observation 
#then only we will get 61st observation as output. so (60)
#1258 last observation in training data
# We want to append 1st 6o observation that means previous 60 stock price
# 0 is the index of column in training_set_scaled dataset

In [9]:
# First valuse of i is 60
# Using above formula = i-60:i, 0
# 60 - 60 : 60 = 0 : 60  i.e 0 to 60 values i.e 0th observation to 59th observation
# i =61 = 61-60 : 61 = 1: 61 Here we will have 59 observation in common and so on

In [10]:
# [i, 0] as i is 60, it will take 60th observation as predict output for 0-59 observation timesteps
# Basically it is T+1 observation as output

In [11]:
# Now we have to build a  data struture
# We have to make this X_train and Y_train to Numpy array

In [12]:
X_train, y_train = np.array(X_train), np.array(y_train)
print(X_train)
print(y_train)

[[0.08581368 0.09701243 0.09433366 ... 0.07846566 0.08034452 0.08497656]
 [0.09701243 0.09433366 0.09156187 ... 0.08034452 0.08497656 0.08627874]
 [0.09433366 0.09156187 0.07984225 ... 0.08497656 0.08627874 0.08471612]
 ...
 [0.92106928 0.92438053 0.93048218 ... 0.95475854 0.95204256 0.95163331]
 [0.92438053 0.93048218 0.9299055  ... 0.95204256 0.95163331 0.95725128]
 [0.93048218 0.9299055  0.93113327 ... 0.95163331 0.95725128 0.93796041]]
[0.08627874 0.08471612 0.07454052 ... 0.95725128 0.93796041 0.93688146]


In [13]:
# This is by using only "open column" we can use diff other column by doing same data preprocessing based on requirnment

## Reshaping the dataset
### - Adding some more dimensionality to previous data struture
### - "open" is only indicator we are using we can use more indicators for better prediction
## - We use Reshape function not only to add more indicators or dimesionality
## - But also to get suitable input format for RNN

# - 3D tensor with shape (batch_size, timesteps, input_dim)
#### using keras documentation we got this

In [14]:
### - This the input shape what RNN wants
### - batch_size = Number of observation i.e number of stock prices
### - timestep = no of time steps
### - input_dim = no of indicators we want or using

In [17]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_train = Dataset we want to reshape
# X_train.shape[0] will give no of observation from X_train data that is batch_size because [0] = rows
# X_train.shape[1] will give no of time steps from X_train data that is [1] = columns
# 1 no of indicator we are using "open"
# One dimension for no of stock price 
# SEcond dimension for time steps
# Third dimension for no of indicators or predictors
X_train

array([[[0.08581368],
        [0.09701243],
        [0.09433366],
        ...,
        [0.07846566],
        [0.08034452],
        [0.08497656]],

       [[0.09701243],
        [0.09433366],
        [0.09156187],
        ...,
        [0.08034452],
        [0.08497656],
        [0.08627874]],

       [[0.09433366],
        [0.09156187],
        [0.07984225],
        ...,
        [0.08497656],
        [0.08627874],
        [0.08471612]],

       ...,

       [[0.92106928],
        [0.92438053],
        [0.93048218],
        ...,
        [0.95475854],
        [0.95204256],
        [0.95163331]],

       [[0.92438053],
        [0.93048218],
        [0.9299055 ],
        ...,
        [0.95204256],
        [0.95163331],
        [0.95725128]],

       [[0.93048218],
        [0.9299055 ],
        [0.93113327],
        ...,
        [0.95163331],
        [0.95725128],
        [0.93796041]]])

# Part 2 - Building the RNN

In [16]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [18]:

# Initialising the RNN
regressor = Sequential()

In [21]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
# units = No of LSTM cells or Neurons in first LSTM Layer
# return_sequence = Used when we are building stacked(Many LSTM Layers) Lstm. We have to keep it true for building next Lstm layer
# And false or leave default for las LSTM layer
# input_shape = Shape of Input giving in First LSTM layer. Here we have to mention only timesteps and Indicators
# No need to mention batch_size(No of observation ) as it will be given during fitting the model to dataset. 
# Dropout to avoid overfitting by ignoring two 20% of neurons for each layer

In [22]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [23]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [24]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [25]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [28]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [1]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

NameError: name 'regressor' is not defined

# Part 3 - Making the predictions and visualising the results

In [16]:
# Getting the real stock price of 2017
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values
# This is a test data i.e tock prices of Jan 
len(dataset_test)

20

In [17]:
# As we have used 60 previous stock price to predict 61st observation , we need to do same in test data from taking 60 previous
# Observation from training data
#Here we need both training set and test set
#Inorder to predict 2nd to last observation of test data we need that remaining previous test data to predict
#

### So we need to Concatinate training and test set

In [18]:
# We must not scale actual test values. 
# We will no concatinate the caled training values
# We will concatinate the original(Not scaled) Values to test values
#  we will get 60 previous values of stock price to predict and we will scale these values

In [19]:
# we have to scale this because we have built our RNN on this scaled values. We must use same scaling = "NORMALIZATION"

In [20]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
# 1st argument = what we want to concatinate which column"[open]""(dataset_train['Open'], dataset_test['Open']"
# 2nd argumnet = concatinate rows or columns we want to concatinate ROWS so "0"
# Now we got all values fro 2012 - 2017 of open column
len(dataset_total)

1278

In [23]:
# Now we want previous 60 stock prices from 1st observation of jan 2017
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
len(inputs)
# 1278 - 20 - 60 : 
# Answer = from 1198th observation to last observation " :"
inputs

array([779.  , 779.66, 777.71, 786.66, 783.76, 781.22, 781.65, 779.8 ,
       787.85, 798.24, 803.3 , 795.  , 804.9 , 816.68, 806.34, 801.  ,
       808.35, 795.47, 782.89, 778.2 , 767.25, 750.66, 774.5 , 783.4 ,
       779.94, 791.17, 756.54, 755.6 , 746.97, 755.2 , 766.92, 771.37,
       762.61, 772.63, 767.73, 764.26, 760.  , 771.53, 770.07, 757.44,
       744.59, 757.71, 764.73, 761.  , 772.48, 780.  , 785.04, 793.9 ,
       797.4 , 797.34, 800.4 , 790.22, 796.76, 795.84, 792.36, 790.9 ,
       790.68, 793.7 , 783.33, 782.75, 778.81, 788.36, 786.08, 795.26,
       806.4 , 807.86, 805.  , 807.14, 807.48, 807.08, 805.81, 805.12,
       806.91, 807.25, 822.3 , 829.62, 837.81, 834.71, 814.66, 796.86])

In [22]:
1278 - 1198

80

In [25]:
inputs = inputs.reshape(-1,1)
# To shape it as Numpy array
inputs

array([[779.  ],
       [779.66],
       [777.71],
       [786.66],
       [783.76],
       [781.22],
       [781.65],
       [779.8 ],
       [787.85],
       [798.24],
       [803.3 ],
       [795.  ],
       [804.9 ],
       [816.68],
       [806.34],
       [801.  ],
       [808.35],
       [795.47],
       [782.89],
       [778.2 ],
       [767.25],
       [750.66],
       [774.5 ],
       [783.4 ],
       [779.94],
       [791.17],
       [756.54],
       [755.6 ],
       [746.97],
       [755.2 ],
       [766.92],
       [771.37],
       [762.61],
       [772.63],
       [767.73],
       [764.26],
       [760.  ],
       [771.53],
       [770.07],
       [757.44],
       [744.59],
       [757.71],
       [764.73],
       [761.  ],
       [772.48],
       [780.  ],
       [785.04],
       [793.9 ],
       [797.4 ],
       [797.34],
       [800.4 ],
       [790.22],
       [796.76],
       [795.84],
       [792.36],
       [790.9 ],
       [790.68],
       [793.7 ],
       [783.33

In [ ]:
# Scaling values to above SC object
inputs = sc.transform(inputs)
# Here we must not use fit method again
# We must just use transform because it will scale same as it scaled training set
# so both scaled values will be same

In [ ]:
# Create empty entity
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)

# From inputs append previous 60th observation to ith observation


In [ ]:
# Reshaping same as above as 3D input for RNN
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# MAke prediction using predict method
predicted_stock_price = regressor.predict(X_test)

In [ ]:
# Now the predicted values are in scaled format
# So we have to bring those scaled values back to origonal values
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualizing the prediction

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')# Plotting real stock price with predicted stock price
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()